In [ ]:
import tensorflow as tf
import os
import numpy as np
import tensorflow_datasets as tfds

# Check if TPU is set as hardware accelerator and get tpu address

In [ ]:
assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

tpu_address = TF_MASTER

# Initialize the TPU

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
print("Number of devices: ", len(tf.config.list_logical_devices('TPU')))

INFO:tensorflow:Initializing the TPU system: grpc://10.108.236.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.108.236.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]
Number of devices:  8


# Choose a distribution strategy for the hardware accelerator
# - Run the model on multiple TPUs in a data parallel way

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Create a Convolutional Neural Network and train it on TPUs

In [ ]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, (3,3), activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)
       ])

In [ ]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(32, (5,5), activation='relu', padding='same', input_shape=(28, 28, 1)),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
       tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(1024, activation='relu'),
       tf.keras.layers.Dropout(0.4),
       tf.keras.layers.Dense(10)
       ])

# Load in the MNIST Dataset

In [ ]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split, with_info=True,
                            as_supervised=True, try_gcs=True)

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

  dataset = dataset.map(scale)

  # Only shuffle and repeat the dataset in training. The advantage to have a
  # infinite dataset for training is to avoid the potential last partial batch
  # in each epoch, so users don't need to think about scaling the gradients
  # based on the actual batch size.
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

# Compile the model

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])
  
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 256)       2560      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 256)       590080    
_________________________________________________________________
flatten (Flatten)            (None, 147456)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               37748992  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 38,375,818
Trainable params: 38,375,818
Non-trainable params: 0
____________________________________________

# Train the model on TPUs

In [ ]:
batch_size = 200

train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

In [ ]:
epochs = 5
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

history = model.fit(train_dataset,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=test_dataset, 
                    validation_steps=validation_steps)

Epoch 1/5
300/300 [==============================] - 21s 44ms/step - loss: 0.3620 - sparse_categorical_accuracy: 0.8913 - val_loss: 0.0422 - val_sparse_categorical_accuracy: 0.9872
Epoch 2/5
300/300 [==============================] - 10s 34ms/step - loss: 0.0389 - sparse_categorical_accuracy: 0.9879 - val_loss: 0.0485 - val_sparse_categorical_accuracy: 0.9847
Epoch 3/5
300/300 [==============================] - 10s 35ms/step - loss: 0.0232 - sparse_categorical_accuracy: 0.9929 - val_loss: 0.0520 - val_sparse_categorical_accuracy: 0.9850
Epoch 4/5
300/300 [==============================] - 11s 38ms/step - loss: 0.0151 - sparse_categorical_accuracy: 0.9951 - val_loss: 0.0367 - val_sparse_categorical_accuracy: 0.9901
Epoch 5/5
300/300 [==============================] - 11s 36ms/step - loss: 0.0079 - sparse_categorical_accuracy: 0.9977 - val_loss: 0.0511 - val_sparse_categorical_accuracy: 0.9870


In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                # Anything between 2 and `steps_per_epoch` could help here.
                steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

epochs = 5
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

history = model.fit(train_dataset,
                epochs=5,
                steps_per_epoch=steps_per_epoch,
                validation_data=test_dataset,
                validation_steps=validation_steps)

Epoch 1/5
300/300 [==============================] - 15s 51ms/step - loss: 0.2203 - sparse_categorical_accuracy: 0.9332 - val_loss: 0.0466 - val_sparse_categorical_accuracy: 0.9853
Epoch 2/5
300/300 [==============================] - 5s 18ms/step - loss: 0.0359 - sparse_categorical_accuracy: 0.9889 - val_loss: 0.0442 - val_sparse_categorical_accuracy: 0.9860
Epoch 3/5
300/300 [==============================] - 5s 18ms/step - loss: 0.0193 - sparse_categorical_accuracy: 0.9940 - val_loss: 0.0395 - val_sparse_categorical_accuracy: 0.9885
Epoch 4/5
300/300 [==============================] - 6s 18ms/step - loss: 0.0141 - sparse_categorical_accuracy: 0.9953 - val_loss: 0.0527 - val_sparse_categorical_accuracy: 0.9847
Epoch 5/5
300/300 [==============================] - 5s 18ms/step - loss: 0.0093 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.0530 - val_sparse_categorical_accuracy: 0.9870


# Visualization of Training Process

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot the accuracy of the model during training
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Sparse_categorical_accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot the loss of the model during training
plt.plot(history.history['loss'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Evaluate the best model with testing data.
 evaluation = model.evaluate(test_dataset)
 print(evaluation)